In [1]:
import json

In [2]:
with open(r"odis_json\64814.json", mode="r", encoding="utf-8") as read_file:
    file_data = json.load(read_file)

In [12]:
# Extract match info directly
match_summary = file_data["info"]  # Keeping the original structure

# Initialize innings summary
innings_summary = []

for inning in file_data["innings"]:
    team = inning["team"]
    total_runs = 0
    total_wickets = 0
    total_extras = 0
    batsmen_stats = {}
    bowlers_stats = {}

    for over in inning["overs"]:
        for delivery in over["deliveries"]:
            batter = delivery["batter"]
            bowler = delivery["bowler"]
            runs = delivery["runs"]["total"]
            extras = delivery["runs"]["extras"]
            total_runs += runs
            total_extras += extras

            # Batting stats
            if batter not in batsmen_stats:
                batsmen_stats[batter] = {"runs": 0, "balls": 0, "strike_rate": 0}
            batsmen_stats[batter]["runs"] += delivery["runs"]["batter"]
            batsmen_stats[batter]["balls"] += 1

            # Bowling stats
            if bowler not in bowlers_stats:
                bowlers_stats[bowler] = {"runs_given": 0, "wickets": 0, "balls": 0}
            bowlers_stats[bowler]["runs_given"] += runs
            bowlers_stats[bowler]["balls"] += 1

            # Wicket stats
            if "wickets" in delivery:
                total_wickets += 1
                if bowler in bowlers_stats:
                    bowlers_stats[bowler]["wickets"] += 1

    # Compute strike rate for batsmen
    for batter, stats in batsmen_stats.items():
        if stats["balls"] > 0:
            stats["strike_rate"] = round((stats["runs"] / stats["balls"]) * 100, 2)

    # Compute economy rate for bowlers
    for bowler, stats in bowlers_stats.items():
        overs_bowled = stats["balls"] / 6
        stats["overs"] = round(overs_bowled, 1)
        if overs_bowled > 0:
            stats["economy"] = round(stats["runs_given"] / overs_bowled, 2)

    # Find top performers
    top_scorer = max(batsmen_stats.items(), key=lambda x: x[1]["runs"], default=(None, {}))
    best_bowler = max(bowlers_stats.items(), key=lambda x: x[1]["wickets"], default=(None, {}))

    innings_summary.append({
        "team": team,
        "total_runs": total_runs,
        "total_wickets": total_wickets,
        "total_extras": total_extras,
        "top_scorer": {"player": top_scorer[0], "runs": top_scorer[1].get("runs", 0)},
        "best_bowler": {"player": best_bowler[0], "wickets": best_bowler[1].get("wickets", 0)},
        "batting_contributions": batsmen_stats,
        "bowling_contributions": bowlers_stats
    })

# Combine match summary and innings summary
match_scorecard = {
    "match_summary": match_summary,  # Keeping "info" as it is
    "innings_summary": innings_summary
}

# Save the summary as JSON
output_file = "match_summary_with_players.json"  # Change this to your desired output path
with open(output_file, "w") as f:
    json.dump(match_scorecard, f, indent=4)

print(f"Match summary JSON file created: {output_file}")


Match summary JSON file created: match_summary_with_players.json
